### Imports

In [ ]:
import numpy as np
import shap
import matplotlib.pyplot as plt
import pandas as pd
import ydata_profiling as yp

from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from collections import Counter

import os
import sys
import json
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.manifold import TSNE
from scipy.stats import linregress

NO_INFERENCE_METRICS = [
    "shape_x",
    "shape_y",
    "shape_z",
    "noise",
    "blur",
    "edge_strength",
    "snr",
    "variance",
    "otsu",
    "entropy",
    "haziness",  # "haziness2", "weber_contrast",
    # "vol_true",
    # "axis_major_length",
    # "axis_minor_length",
    # "extent",
    "intensity_max",
    "intensity_mean",
    "intensity_min",
    # "intensity_std",
    # "intensity_diff",
    # "orientation",
    # "perimeter",
    # "solidity",
]

plt.rcParams["font.family"] = ["sans-serif"]

### Load segmentation metrics report

In [ ]:
pwd = os.getcwd()


df2 = pd.read_excel(os.path.join(pwd, "generic_dataset.xlsx"))
# df_all = pd.concat([df, df2])
df_all = df2.copy()

# Handling missing values
df_all.fillna(0, inplace=True)

# Removing duplicate rows
# df_all.drop_duplicates(inplace=True)
df_all.infer_objects()

### Compute metric report statistics

In [ ]:
# Calculating summary statistics
# summary_stats = df_all.describe().T
# summary_stats.to_excel("results/metric_stats_report.xlsx")
# df_all.describe().T

### Strip data down to pre-inference features only

In [ ]:
df_all = df_all.drop(["id", "balanced_accuracy", "iou_jaccard"], axis=1)
df_all = df_all.drop(["tn", "tp", "fn", "fp", "tpr", "fnr", "fpr", "ssim"], axis=1)
df_all = df_all.drop(["tnr", "ppv", "npv", "f1", "accuracy"], axis=1)
df_all = df_all.drop(["area", "area_bbox"], axis=1)
df_all.head()

### Feature numpy stacking and scaling

In [ ]:
features = []
feature_names = []
yscores = df_all["dice"].to_numpy()
# ymean = np.mean(yscores)
# yscores = 1*(yscores>ymean)
for col in df_all.columns:
    if col in NO_INFERENCE_METRICS:
        if df_all[col].dtype == "object":
            df_all[col] = df_all[col].astype("category")
            df_all[col] = df_all[col].cat.codes.astype(np.uint8)
            df2[col + "_code"] = df_all[col]
        feature_names.append(col)
        features.append(df_all[col].to_numpy())

df2.to_excel("generic_dataset_final_features.xlsx", index=False)
feature_names = dict(enumerate(feature_names))
Xfeatures = np.array(features).T
print(Xfeatures.shape, yscores.shape)

Xfeatures = np.nan_to_num(Xfeatures, nan=0.0, posinf=0.0, neginf=0.0)
Xfeatures_unscaled = Xfeatures.copy()
Xfeatures = MinMaxScaler().fit_transform(Xfeatures)

df_minmax = pd.DataFrame(Xfeatures, columns=feature_names.values())

print(feature_names.values())

### PCA

In [ ]:
pca = PCA(n_components=2)
pca_features = pca.fit_transform(Xfeatures)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
plt.scatter(pca_features[:, 0], pca_features[:, 1], c=yscores, cmap="plasma", alpha=0.8, s=12)
plt.xlabel("PCA projection 1")
plt.ylabel("PCA projection 2")
plt.colorbar()
plt.pause(0.001)

### t-SNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=30.0, early_exaggeration=12.0, max_iter=2000, min_grad_norm=1e-07, init="random")  # , init='pca')
tsne_features = tsne.fit_transform(Xfeatures)

fig, ax = plt.subplots()
sc = plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=yscores, cmap="plasma", alpha=0.8, s=12)
plt.xlabel("t-SNE embedding 1")
plt.ylabel("t-SNE embedding 2")
plt.colorbar()

plt.pause(0.001)

### Random forest feature importance

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(Xfeatures, yscores)
rf_train = rf_model.score(Xfeatures, yscores)
rf_cv = cross_val_score(rf_model, Xfeatures, yscores, cv=10).mean()

print(f"Random Forest training score: {np.round(rf_train, 4)}")
print(f"Random Forest cross validation score: {np.round(rf_cv, 4)}")

importances = rf_model.feature_importances_
stds = np.var([m.feature_importances_ for m in rf_model.estimators_], axis=0)
rf_importance_df = pd.DataFrame({"Feature": feature_names.values(), "Importance": importances})
rf_importance_df.sort_values(by="Importance", ascending=False, inplace=True)

print("Random Forest Top 7 Features:")
for i, (_, f) in enumerate(rf_importance_df["Feature"].head(7).items()):
    print(i + 1, f)

plt.bar(rf_importance_df["Feature"], rf_importance_df["Importance"], yerr=stds)
plt.xticks(rotation=90)
# plt.title("Random Forest Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Random Forest Feature Importance")
plt.pause(0.001)

### Can these features predict dice score outcome?

In [ ]:
pred = rf_model.predict(Xfeatures)
m, b, r_value, p_value, std_err = linregress(yscores, pred)
eq = f"$y = {m:.3f}x + {b:.3f}$"
r_squared = f"$R^2 = {r_value**2:.3f}$"

plt.scatter(yscores, pred, s=12)
plt.plot(yscores, m * yscores + b, c="red", linestyle=":", label=eq + "\n" + r_squared)
plt.xlabel("Actual Dice")
plt.ylabel("Feature-inferred Dice")
plt.title("Feature-inferred Dice vs Actual Dice")
plt.legend()
plt.pause(0.001)

### Recursive Feature elimination (RFE) feature importance

In [ ]:
used = []
for x in range(Xfeatures.shape[1]):
    selector = RFE(rf_model, n_features_to_select=x + 1, step=1)
    selector = selector.fit(Xfeatures, yscores)

    selected_features = [feature_names[i] for i in range(len(feature_names)) if selector.support_[i] and feature_names[i] not in used]
    used.append(selected_features[0])
    print(f"top {x + 1} features: {selected_features}")
    # print("Recursive Feature elimination (RFE) Top 15 Features:")
    # for i in range(15):
    #     print(selected_features[i])
print(used)
rfe_importance_df = pd.DataFrame({"Feature": used, "Importance": range(len(used), 0, -1)})
rfe_importance_df.sort_values(by="Importance", ascending=False, inplace=True)
plt.bar(rfe_importance_df["Feature"], rfe_importance_df["Importance"])
plt.xticks(rotation=90)
plt.xlabel("Feature")
plt.ylabel("RFE Feature Importance")
plt.pause(0.001)

### Permutation feature importance

In [ ]:
presults = permutation_importance(rf_model, Xfeatures, yscores, n_repeats=30)

perm_importance_df = pd.DataFrame({"Feature": feature_names.values(), "Importance": presults.importances_mean})
perm_importance_df.sort_values(by="Importance", ascending=False, inplace=True)

print("Permutation Top 15 Features:")
for i, (_, f) in enumerate(perm_importance_df["Feature"].head(15).items()):
    print(i + 1, f)

plt.bar(perm_importance_df["Feature"], perm_importance_df["Importance"], yerr=presults.importances_std)
plt.xticks(rotation=90)
plt.xlabel("Feature")
plt.ylabel("Permutation Feature Importance")
# plt.title('Permutation Feature Importance')
plt.pause(0.001)

### XGBoost feature importance

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(Xfeatures, yscores)

importances = xgb_model.feature_importances_
xg_importance_df = pd.DataFrame({"Feature": feature_names.values(), "Importance": importances})
xg_importance_df.sort_values(by="Importance", ascending=False, inplace=True)

print("XGBoost Top 15 Features:")
for i, (_, f) in enumerate(xg_importance_df["Feature"].head(15).items()):
    print(i + 1, f)

plt.bar(xg_importance_df["Feature"], xg_importance_df["Importance"])
plt.xticks(rotation=90)
# plt.title("XGBoost Feature Importance")
plt.xlabel("XGBoost Feature Importance")
plt.ylabel("Importance")
plt.pause(0.001)

### SHAP feature importance

In [ ]:
# np.obj2sctype =  lambda obj: np.dtype(obj).type
shap.initjs()

explainer = shap.Explainer(xgb_model)
shap_df = pd.DataFrame(Xfeatures, columns=list(feature_names.values()))

# shap bee-swarm plot
shap_test = explainer(shap_df)
shap_importance_df = shap_df.apply(np.abs).mean().sort_values(ascending=False)
print(shap_importance_df.head())
shap.plots.beeswarm(shap_values=shap_test, max_display=15)

# shap force plot
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(Xfeatures)
shap.force_plot(explainer.expected_value, shap_values[0, :], list(feature_names.values()))

In [ ]:
ordered_importance_dicts = []
for df in rf_importance_df, rfe_importance_df, perm_importance_df, xg_importance_df:
    ordered_importance_dicts.append(dict(enumerate(list(df["Feature"]), start=1)))

feature_dict = dict(zip(feature_names.values(), np.zeros(len(feature_names))))
for d in ordered_importance_dicts:
    for name in feature_dict.keys():
        feature_dict[name] += list(d.keys())[list(d.values()).index(name)]

sorted_feature_dict = dict(sorted(feature_dict.items(), key=lambda item: item[1]))

sorted_features = list(sorted_feature_dict.keys())
print("Overall Top 7 Features:")
for i, feat in enumerate(sorted_features[:7]):
    print(i + 1, feat)

In [ ]:
N = 5
for i, feat in enumerate(sorted_features[:N]):
    print(i + 1, feat)
    plt.figure(figsize=(10, 5))
    if feat not in ["categorical1", "categorical1"]:  # if column data is numeric
        if "int" in str(df2[feat].dtype):
            this_df = df2.copy()
            quant_df = this_df[feat].quantile([0.5])  # [0.25, 0.5, 0.75])
            # quant_df = this_df[feat].quantile([0.333, 0.667]) #[0.25, 0.5, 0.75])
            # quant_df[0.5] = (this_df[feat].max()-this_df[feat].min())/2
            # quant_df[0.25] = (quant_df[0.5] - this_df[feat].min())/2
            # quant_df[0.75] = (quant_df[0.5] + this_df[feat].max())/2
            quant_df = quant_df.astype(int)
        if "float" in str(df2[feat].dtype):
            this_df = df2.copy()
            quant_df = this_df[feat].quantile([0.5])  # [0.25, 0.5, 0.75])
            # quant_df = this_df[feat].quantile([0.333, 0.667]) #[0.25, 0.5, 0.75])
            # quant_df[0.5] = (this_df[feat].max()-this_df[feat].min())/2
            # quant_df[0.25] = (quant_df[0.5] - this_df[feat].min())/2
            # quant_df[0.75] = (quant_df[0.5] + this_df[feat].max())/2

        # data_ranges = [(-1e100, quant_df[0.25]), (quant_df[0.25], quant_df[0.5]), (quant_df[0.5], quant_df[0.75]), (quant_df[0.75], 1e100)]
        data_ranges = [(-1e100, quant_df[0.5]), (quant_df[0.5], 1e100)]
        # data_ranges = [(-1e100, quant_df[0.333]), (quant_df[0.333], quant_df[0.667]), (quant_df[0.667], 1e100)]

        if "float" in str(df2[feat].dtype):
            quant_df = quant_df.round(3)
        # labels = [f"≤ Q1:\n{quant_df[0.25]}", f"Q1-Q2:\n({quant_df[0.25]}, {quant_df[0.5]}]", f"Q2-Q3:\n({quant_df[0.5]}, {quant_df[0.75]}]", f"> Q3:\n{quant_df[0.75]}"]
        labels = [f"≤ Q2:\n{quant_df[0.5]}", f"> Q2:\n{quant_df[0.5]}"]
        # labels = [f"≤ q1/3:\n{quant_df[0.333]}", f"q1/3-q2/3:\n({quant_df[0.333]}, {quant_df[0.667]}]", f"> q2/3:\n{quant_df[0.667]}"]
        data_groups = []
        df2[feat + "_code"] = np.nan
        for jj in range(len(labels)):
            this_range = data_ranges[jj]
            data_block = (this_df[feat] <= this_range[1]) * (this_df[feat] > this_range[0])
            data_groups.append(list(df2["dice"][data_block]))
            df2.loc[df2.index[data_block].tolist(), feat + "_code"] = int(jj + 1)

        plt.subplot(1, 2, 1)
        plt.boxplot(data_groups, tick_labels=labels, showmeans=True)
        plt.xticks(rotation=90)
        plt.title(f"{feat}, rank {i + 1}")
        plt.ylabel("Dice")

        plt.subplot(1, 2, 2)
        plt.bar(labels, [len(g) for g in data_groups])
        plt.ylabel("Count")
        plt.xticks(rotation=90)
        plt.title(f"{feat}, rank {i + 1}")
        plt.pause(0.001)

    else:  # if column data is non-numeric "object"
        labels = []
        data_groups = []
        # case_groups = []

        df2[feat] = df2[feat].fillna("Unknown")
        # print( np.unique(df2[feat]) )
        df2[feat + "_code"] = np.nan
        for jj, val in enumerate(np.unique(df2[feat])):
            labels.append(val)
            data_groups.append(list(df2["dice"][df2[feat] == val]))
            df2.loc[df2.index[df2[feat] == val].tolist(), feat + "_code"] = int(jj + 1)

        plt.subplot(1, 2, 1)
        plt.boxplot(data_groups, tick_labels=labels, showmeans=True)
        plt.xticks(rotation=90)
        plt.title(f"{feat}, rank {i + 1}")
        plt.ylabel("Dice")

        plt.subplot(1, 2, 2)
        plt.bar(labels, [len(g) for g in data_groups])
        plt.ylabel("Count")
        plt.xticks(rotation=90)
        plt.title(f"{feat}, rank {i + 1}")
        plt.pause(0.001)

top_n = [sorted_features[n] + "_code" for n in range(N)]
code_array = df2[top_n].to_numpy()  # .astype(int)


final_codes = []
for row in range(code_array.shape[0]):
    code = 0
    for i, val in enumerate(code_array[row, :]):
        code += val * (10 ** (N - i - 1))
    final_codes.append(int(code))
df2["final_code"] = final_codes

df2.to_excel("generic_dataset_final_features_coded.xlsx", index=False)

### Train/test/val split based on stratification

In [ ]:
dff = pd.read_excel("generic_dataset_final_features_coded.xlsx")

code_counts = Counter(final_codes)
# print(code_counts)
low_counts = [code for code, count in code_counts.items() if count <= 3]

# print(low_counts)
for i, item in enumerate(dff["final_code"]):
    if item in low_counts:
        dff.loc[i, "final_code"] = int("".join(map(str, [N] * N)))

final_codes = dff["final_code"].to_list()
code_counts = Counter(final_codes)
print(code_counts)

data_len = dff.shape[0]
test_len = 0.14 * data_len
val_len = 0.08 * data_len
X_temp, X_test, y_temp, y_test = train_test_split(dff, dff["final_code"], test_size=test_len / data_len, random_state=42, stratify=dff["final_code"])

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_len / (data_len - test_len), random_state=42, stratify=y_temp)

del X_temp, y_temp, dff
print("TRAIN", X_train.shape, y_train.shape)
print("VAL", X_val.shape, y_val.shape)
print("TEST", X_test.shape, y_test.shape)

dataset_dict = {}
for df, name in zip([X_train, X_val, X_test], ["train", "validation", "test"]):
    cases = df["id"].to_list()
    data_list = []
    for c in cases:
        case_dict = {"image": f"/data/images/{c}.mha", "label": f"/data/labels/{c}.seg.nrrd"}
        data_list.append(case_dict)
    dataset_dict[name] = data_list

with open("generic_dataset_stratified_data_splits.json", "w") as f:
    json.dump(dataset_dict, f, indent=2)

### Generate Feature Profile Report

In [ ]:
sys.exit()
report = yp.ProfileReport(
    df_all,
    title="<ML Task/Project name> Profiling Report",
    dataset={
        "description": "This profiling report was generated using ...",
        "model": "path/to/model.pth",
        "author": "Jim Bob",
        "copyright_holder": "Jim Bob",
        "copyright_year": 2025,
    },
    explorative=True,
)

report.config.interactions.targets = ["dice"]
report.to_file("Generic_dataset_project_name_Profiling_Report_Final.html")
report